In [1]:
from segmentation.scr.utils.utils import set_seed
from segmentation.scr.utils.metrics import dice_coef
from segmentation.models.smp import get_pretrained_model
from segmentation.config import CFG
import torch
import torch.nn as nn
import numpy as np

from segmentation.scr.train_model.data_loader import *
from segmentation.scr.utils.utils import set_seed
from segmentation.scr.train_model.evaluate_dataset import evaluate_dataset




In [2]:
class CFG_3:
    # ============== dice metrics ============
    
    target_size = 1
    in_chans = 3  # 65
    # ============== training CFG =============
    image_size = 512
    input_size = 512
    device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
path_to_model_3_kaag = 'weight/mobnet_3_ch.pth'

#ckpt = torch.load(path_to_model_3_kaag)
#ckpt = ckpt["model"]
#torch.save(ckpt, 'weight/mobnet_3_ch.pth')
model_3_kaag = get_pretrained_model(path_to_model=path_to_model_3_kaag, train_parallel=True, CFG=CFG_3)


In [4]:
path_to_model_3_kaag = 'C:\\Users\\123\\Desktop\\resnet baseline\\state efficientnet-b7_best_model_at_22.pth'

ckpt = torch.load(path_to_model_3_kaag)
ckpt = ckpt["model"]
torch.save(ckpt, 'weight/mobnet_1_5_ch_kaagle.pth')

In [5]:
model_my_pc= get_pretrained_model(path_to_model='weight\mobnet_my_pc.pth', train_parallel=False, CFG=CFG)
model_5_k = get_pretrained_model(path_to_model='weight/mobnet_5_ch_kaagle.pth', train_parallel=True, CFG=CFG)
model_1_5_k = get_pretrained_model(path_to_model='weight/mobnet_1_5_ch_kaagle.pth', train_parallel=True, CFG=CFG)

In [6]:
path_img_dir="data\\train\\kidney_3_sparse\\images"
path_lb_dir="data\\train\\kidney_3_dense\\labels"
img_path, lb_path = create_img_lb_paths(path_img_dir = path_img_dir, path_lb_dir = path_lb_dir)

Общее число изображений с масками сегментации : 501


In [54]:
11 % 3

2

In [58]:
15 + (3 - 15% 3)

18

In [55]:
11 // 3

3

In [8]:
lb_path[500]

'data\\train\\kidney_3_dense\\labels\\0996.tif'

In [9]:
val_x=load_data(img_path ,is_label=False)
print(val_x.shape)
val_y=load_data(lb_path ,is_label=True)
print(val_y.shape)

100%|██████████| 32/32 [00:05<00:00,  5.94it/s]


torch.Size([501, 1706, 1510])


100%|██████████| 32/32 [00:05<00:00,  5.90it/s]

torch.Size([501, 1706, 1510])


In [10]:
set_seed(42)
val_dataset_3=Kaggld_Dataset([val_x],[val_y], arg=False, CFG=CFG_3)
val_loader_3 = DataLoader(val_dataset_3, batch_size=CFG.valid_batch_size, shuffle=False, )

val_dataset_5=Kaggld_Dataset([val_x],[val_y], arg=False, CFG=CFG)
val_loader_5 = DataLoader(val_dataset_5, batch_size=CFG.valid_batch_size, shuffle=False, )

In [15]:
def tta(x, model):
    model.eval()
    x_n=[torch.rot90(x,k=i,dims=(-2,-1)) for i in range(4)]

    shape = x.shape
    for i in range(4):
        if i ==0:
            x_n[i] = x_n[i]
        else:
            x_n[i]=torch.flip(x_n[i] , dims = (-2,))
    with torch.no_grad():
        pred = [model(b) for b in x_n]
    
        torch.flip(pred[i] , dims = (-2,))
    pred=torch.cat(pred,dim=0)
    pred = pred.sigmoid()
    pred =pred.reshape(4,shape[0],*shape[2:])
    
    
    for i in range(4):
        if i ==0:
            pred[i] = pred[i]
        else:
            pred[i]=torch.flip(pred[i] , dims = (-2,))
    pred=[torch.rot90(pred[i],k=-i,dims=(-2,-1)) for i in range(4)]
    pred=torch.stack(pred,dim=0).mean(0)
            

    return pred

In [41]:
def create_tilling( images, overlap_pct = 50):
    if len(images.shape) == 2:
        #print('ss')
        images = images.unsqueeze(0).repeat(CFG.in_chans,1,1)
    min_overlap = float(overlap_pct) * 0.01
    max_stride = CFG.image_size * (1.0 - min_overlap)

    #print(images.shape)
    height, width = images.shape[1], images.shape[2]
    num_patches = np.ceil(np.array([height, width]) / max_stride).astype(
                np.int32
            )

    starts = [
                np.int32(np.linspace(0, height - CFG.input_size, num_patches[0])),
                np.int32(np.linspace(0, width - CFG.input_size, num_patches[1])),
            ]
    stops = [starts[0] + CFG.input_size, starts[1] + CFG.input_size]
        
    indexs=[]
    tills=[]
    for y1, y2 in zip(starts[0], stops[0]):
        for x1, x2 in zip(starts[1], stops[1]):
            tills.append(images[...,y1:y2, x1:x2])
            indexs.append((y1,y2,x1,x2))
                
  
    return tills, indexs

In [45]:
def evaluate_dataset_2(model, val_loader):
    model.eval()
    timer = tqdm(range(len(val_loader)))
    val_scores = 0
    scores = []
    for i, (x, y) in enumerate(val_loader):
        x = x.cuda().to(torch.float32)
        y = y.cuda().to(torch.float32)
        x = norm_with_clip(x.reshape(-1, *x.shape[2:])).reshape(x.shape)
        with torch.no_grad():
            pred = tta(model=model, x=x)

        score = dice_coef(pred.detach(), y, from_logits=False)
        scores.append(score.detach().cpu().item())
        val_scores = (val_scores * i + score) / (i + 1)
        timer.set_description(f"eval--> score:{val_scores:.4f}")
        timer.update()
    timer.close()
    return scores, val_scores

In [46]:
scores_2, val_scores_2 = evaluate_dataset_2(model=model_my_pc, val_loader=val_loader_5)

In [53]:
print(val_scores_2, val_scores_my)

tensor(0.8458, device='cuda:0') tensor(0.8389, device='cuda:0')


In [50]:
scores_my, val_scores_my= evaluate_dataset(model=model_my_pc ,val_loader=val_loader_5)

In [52]:
import scipy
scipy.stats.ttest_rel(a=scores_my, b=scores_2)

TtestResult(statistic=-0.4433978925408387, pvalue=0.6582568970975875, df=123)